In [1]:
from source.read_data import MockDB
import pandas as pd
from pprint import pprint
from collections import OrderedDict
import os

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
db = MockDB()

In [4]:
schema = db.schema

In [5]:
schema_legislacao = {key : val for key, val in schema.items() if 'legislation' in key}

In [6]:
pprint(schema_legislacao)

{'legislation_annotations': ['id',
                             'quote',
                             'ranges',
                             'text',
                             'legislation_draft_version_id',
                             'author_id',
                             'hidden_at',
                             'created_at',
                             'updated_at',
                             'comments_count',
                             'range_start',
                             'range_start_offset',
                             'range_end',
                             'range_end_offset',
                             'context',
                             None,
                             None,
                             None,
                             None,
                             None,
                             None,
                             None],
 'legislation_answers': ['id',
                         'legislation_question_id',
                  

In [7]:
def processes_with_name(db):
    
    processess = db['legislation_processes']
    names = db['legislation_process_translations']
    
    merged = pd.merge(
                    names, 
                    processess,
                    left_on = 'legislation_process_id',
                    right_on = 'id',
                    how='inner',
                    suffixes = ('_process_names', '_process')
                     )
    
    return merged

In [8]:
def filter_processes_columns(processes_with_names):
    
    columns = [
        'legislation_process_id',
        'title',
        'summary',
        'start_date',
        'end_date'
    ]
    
    
    return processes_with_names[columns].copy()

In [9]:
def draft_by_process(db, processes):
    
    drafts = db['legislation_draft_versions']
    
    
    merged = pd.merge(
                    drafts, 
                    processes,
                    on = 'legislation_process_id',
                    how='left',
                    suffixes = ('_draft', '_process')
                     )
    return merged

In [10]:
def anotations_by_draft(db, drafts):
    
    anottations = db['legislation_annotations']
    
    merged = pd.merge(
                    drafts, 
                    anottations,
                    left_on = 'id',
                    right_on = 'legislation_draft_version_id',
                    how='right',
                    suffixes = ('_draft', '_anottations')
                     )
    return merged

In [11]:
def anottations_dataset(db, processes):
    
    drafts = draft_by_process(db, processes)
    anottations = anotations_by_draft(db, drafts)
    
    renamed_columns = OrderedDict({
        'legislation_process_id' : 'process_id',
        'title' : 'process_title',
        'summary' : 'process_summary',
        'start_date' : 'process_start_date',
        'end_date': 'process_end_date',
        'process_end_year' : 'process_end_year',
        'id_draft' : 'draft_id',
        'status' : 'draft_status',
        'id_anottations' : 'anottation_id',
        'quote' : 'anottation_quote',
        'text' : 'anottation_text',
        'context' : 'anottation_context',
        'author_id' : 'anottation_author_id'
    })
    
    anottations = anottations.rename(renamed_columns, axis = 1)
    anottations = anottations[renamed_columns.values()].copy()
    
    return anottations

In [12]:
def legis_topics_by_process(db, processes):
    
    topics = db['legislation_topics']
    
    
    merged = pd.merge(
                    topics, 
                    processes,
                    on = 'legislation_process_id',
                    how='left',
                    suffixes = ('_topic', '_process')
                     )
    return merged

In [13]:
def votes_by_topic(db, topics):
    
    votes = db['legislation_topic_votes']
    
    merged = pd.merge(
                    topics, 
                    votes,
                    left_on = 'id',
                    right_on = 'legislation_topic_id',
                    how='right',
                    suffixes = ('_topic', '_votes')
                     )
    return merged

In [14]:
def get_vote_values(db, votes):
    
    asssesments = db['legislation_assessments']
    votes = pd.merge(votes, 
                     asssesments, 
                     how='left',
                     left_on = 'legislation_assessment_id',
                     right_on = 'id',
                     suffixes = ('_vote', '_vote_value')
                    )
    
    return votes

In [15]:
def votes_dataset(db, processes):
    
    topics = legis_topics_by_process(db, processes)
    votes = votes_by_topic(db, topics)
    votes = get_vote_values(db, votes)
    
    renamed_columns = OrderedDict({
        'legislation_process_id' : 'process_id',
        'title_process' : 'process_title',
        'summary' : 'process_summary',
        'start_date' : 'process_start_date',
        'end_date': 'process_end_date',
        'process_end_year': 'process_end_year',
        'id_topic' : 'topic_id',
        'legislation_topic_id_vote' : 'topic_id',
        'title_topic' : 'topic_title',
        'description' : 'topic_description',
        'topic_votes_count_vote' : 'topic_votes_count',
        'id_votes' : 'vote_id',
        'user_id' : 'voter_user_id',
        'comment' : 'voter_comment',
        'title' : 'vote_value'
    })
        
    votes = votes.rename(renamed_columns, axis = 1)
    votes = votes[renamed_columns.values()].copy()
    
    return votes
        
        
    

In [16]:
def proposals_by_process(db, processes):
    
    proposals = db['legislation_proposals']
    
    merged = pd.merge(
                    proposals, 
                    processes,
                    on = 'legislation_process_id',
                    how='left',
                    suffixes = ('_proposal', '_process')
                     )
    return merged

In [17]:
def proposals_data_set(db, processes):
    
    proposals = proposals_by_process(db, processes)
    
    #verificar depois se acrescento votos
    renamed_columns = OrderedDict({
        'legislation_process_id' : 'process_id',
        'title_process' : 'process_title',
        'title_process' : 'process_summary',
        'start_date' : 'process_start_date',
        'end_date': 'process_end_date',
        'process_end_year' : 'process_end_year',
        'id' : 'proposal_id',
        'title_proposal' : 'proposal_title',
        'summary_proposal' : 'proposal_summary',
        'description' : 'proposal_description',
        'author_id' : 'proposal_author_id'
        
    })
    
    proposals = proposals.rename(renamed_columns, axis = 1)
    proposals = proposals[renamed_columns.values()].copy()
    
    return proposals

In [18]:
def questions_by_process(db, processess):
    
    questions = db['legislation_questions']
    
    merged = pd.merge(
                    questions, 
                    processess,
                    on = 'legislation_process_id',
                    how='left',
                    suffixes = ('_questions', '_process')
                     )
    return merged

In [19]:
def answers_by_question(db, questions):
    
    answers = db['legislation_answers']
    
    merged = pd.merge(
                    questions, 
                    answers,
                    left_on = 'id',
                    right_on = 'legislation_question_id',
                    how='right',
                    suffixes = ('_questions', '_answers')
                     )
    return merged

In [20]:
def answers_data_set(db, processes):
    
    questions = questions_by_process(db, processes)
    answers = answers_by_question(db, questions)
    
    renamed_columns = OrderedDict({
        'legislation_process_id' : 'process_id',
        'title' : 'process_title',
        'summary' : 'process_summary',
        'start_date' : 'process_start_date',
        'end_date': 'process_end_date',
        'process_end_year' : 'process_end_year',
        'id_questions' : 'question_id',
        'answers_count' : 'question_answers_count',
        'id_answers' : 'answer_id',
        'author_id' : 'question_author_id',
        'legislation_question_option_id' : 'question_option_id',
        'user_id' : 'answer_author_id'
        
    })
    
    answers = answers.rename(renamed_columns, axis = 1)
    answers = answers[renamed_columns.values()].copy()
    
    return answers

In [21]:
def get_process_end_year(dataset):
    
    
    dataset = dataset.copy()
    
    splited_dates = dataset['end_date'].astype(str).str.split('-')
    year = splited_dates.apply(lambda x: x[0])
    
    dataset['process_end_year'] = year
    
    return dataset

In [22]:
def get_unique_user(data_obj, proc_id):
    
    df = data_obj['data']
    col_user = data_obj['user_key']
    
    df = df[df['process_id']==proc_id].copy()
    
    df = df[df[col_user].notnull()].copy()
    
    return df[col_user].unique()

In [23]:
def get_unique_users_process(dados, proc_id):
    
    unique_users_process = set()
    for data_obj in dados.values():

        unique_users_modality = get_unique_user(data_obj, proc_id)
        
        unique_users_process.update(unique_users_modality)
    
    return unique_users_process

In [24]:
def get_participants_by_process(processes, dados):
    
    result = []
    for i, row in processes.iterrows():
        proc_id = row['legislation_process_id']
        process_data = {}
        process_data['id'] = proc_id
        process_data['title'] = row['title']
        process_data['ano'] = row['process_end_year']
        
        usuarios_unicos = get_unique_users_process(dados, proc_id)
        
        process_data['total_participantes'] = len(usuarios_unicos)
        
        result.append(process_data)
    
    return pd.DataFrame(result)
            
                

In [25]:
processes = processes_with_name(db)
processes = filter_processes_columns(processes)
processes = get_process_end_year(processes)

In [26]:
dados = dict(
        respostas_questoes = 
                dict(data = answers_data_set(db, processes),
                      user_key = 'answer_author_id'),
            propostas = 
                dict(data = proposals_data_set(db, processes),
                     user_key = 'proposal_author_id'),
            revisoes = 
                    dict(data = votes_dataset(db, processes),
                        user_key = 'voter_user_id'),
            anotacoes = dict(data = anottations_dataset(db, processes),
                             user_key = 'anottation_author_id')
            )

In [35]:
dados['revisoes']['data']['vote_value'].unique()

array(['Manter', 'Revisar', 'Excluir', 'Manter e propor novo',
       'Excluir e propor novo', 'Alterar', 'Incluir', 'Não implementado',
       'Implementado', 'Implementado parcialmente', 'Propor Nova'],
      dtype=object)

In [27]:
dados_final = get_participants_by_process(processes, dados)

In [32]:
dados_final

,id,title,ano,total_participantes
0,4,Edital de contratação da Central de Atendiment...,2020,6
1,6,4a Etapa do Projeto Piloto de Arborização de ...,2020,34
2,3,Contratação de solução tecnológica para os can...,2020,9
3,25,Minuta Revisão Participativa do PDE,2013,0
4,23,PIU Vila Olímpia,2017,0
...,...,...,...,...
156,162,Ações 2021-2024 do Plano de Segurança Viária,2021,34
157,163,Plano de Ação para Implementação da Agenda Mun...,2021,25
158,164,Contratação de Meio de Pagamento Integrado de ...,2022,6
159,116,Programa de Metas 2021-2024 e Plano Plurianual...,2021,275


In [28]:
total_ano = dados_final.groupby('ano').sum()[['total_participantes']]

In [29]:
total_ano

,total_participantes
ano,
2013,0
2014,0
2015,0
2016,0
2017,0
2018,0
2019,0
2020,189
2021,2322


In [30]:
dados_final.to_excel(os.path.join('generated_data', 'dados_final.xlsx'), index=False)

In [31]:
total_ano.to_excel(os.path.join('generated_data', 'total_ano.xlsx'), index=False)